# IV.1. Установка пакета h2o

В Python устанавливаем пакет h2o с помощью строки ```pip install h2o```.

# IV.2. Запуск кластера H2O

In [1]:
# импортируем необходимые библиотеки
import h2o
import os
h2o.init(nthreads=-1, max_mem_size=8)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2OConnectionError: Version mismatch. H2O is version 3.24.0.3, but the h2o-python package is version 3.22.1.5. Install the matching h2o-Python version from - http://h2o-release.s3.amazonaws.com/h2o/rel-yates/3/index.html.

# IV.3. Преобразование данных во фреймы H2O

## IV.3.1. Получение фреймов H2O из датафреймов pandas

In [ ]:
# импортируем библиотеки pandas и numpy
import pandas as pd
import numpy as np

In [ ]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm_for_H2O.csv', sep=';')
data.head(3)

In [ ]:
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis=1), 
                                                    data['Response'], 
                                                    test_size=0.3,
                                                    stratify=data['Response'],
                                                    random_state=42)

In [ ]:
# конкатенируем обучающие массив признаков и массив меток
train = pd.concat([X_train, y_train], axis=1)
# конкатенируем тестовые массив признаков и массив меток
test = pd.concat([X_test, y_test], 
                  axis=1)
# преобразовываем датафреймы pandas во фреймы h2o
tr = h2o.H2OFrame(train)
tst = h2o.H2OFrame(test)

## IV.3.2. Получение фреймов H2O напрямую

In [ ]:
# загружаем данные в формате фрейма
data = h2o.upload_file(path='Data/StateFarm_for_H2O.csv') 

In [ ]:
# разбиваем фрейм на обучающий и тестовый
tr, tst = data.split_frame(ratios=[.7], seed=42)

# IV.4. Знакомство с содержимым фрейма

In [ ]:
# смотрим содержимое фрейма
tr.describe()

In [ ]:
# преобразовываем в категориальную переменную
tr['Response'] = tr['Response'].asfactor()
tst['Response'] = tst['Response'].asfactor()

In [ ]:
# смотрим содержимое фрейма
tr.describe()

# IV.5. Определение имени зависимой переменной и списка имен предикторов

In [ ]:
# задаем имя зависимой переменной
dependent = 'Response'
# задаем список имен предикторов
predictors = list(tr.columns)
# удаляем имя зависимой переменной 
# из списка имен предикторов
predictors.remove(dependent)

# IV.6. Обучение модели машинного обучения

In [ ]:
# импортируем класс H2ORandomForestEstimator
from h2o.estimators import H2ORandomForestEstimator
# создаем экземпляр класса H2ORandomForestEstimator
forest_model = H2ORandomForestEstimator(seed=42)
# обучаем модель
forest_model.train(x = predictors, 
                   y = dependent, 
                   training_frame=tr, 
                   validation_frame=tst)

# IV.7. Вывод модели

In [ ]:
# смотрим модель
forest_model

# IV.8. Получение прогнозов

In [ ]:
# получаем спрогнозированные значения и спрогнозированные 
# вероятности классы зависимой переменной
predictions = forest_model.predict(tst)
predictions

In [ ]:
# завершаем работу с H2O
h2o.cluster().shutdown()